In [1]:
from simtk.openmm.app import ForceField, PDBFile, Modeller, Simulation, PME, HBonds, StateDataReporter
from simtk.openmm import app, LangevinIntegrator, MonteCarloBarostat
from simtk.unit import *
from sys import stdout
from mdtraj.reporters import HDF5Reporter

import mdtraj
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#----------------------------------------------------------------------#
# Parameters
eqtime = 500000   # 1ns
prtime = 50000000 # 100ns
#----------------------------------------------------------------------#

In [4]:
ff = ForceField('amber14/protein.ff14SB.xml', 'amber14/tip3p.xml')
pdb = PDBFile("pdz3_rat_apo_fixed.pdb")
modeller = Modeller(pdb.topology, pdb.positions)
#modeller.addSolvent(ff, boxSize=(90., 90., 90.)*angstroms)
system = ff.createSystem(modeller.topology,
                         removeCMMotion=False,
                         nonbondedMethod=app.CutoffNonPeriodic, nonbondedCutoff=1.0*nanometers,
                         constraints=None, rigidWater=True)
#barostat = MonteCarloBarostat(1.0*bar, 277*kelvin, 25)
#system.addForce(barostat)

In [5]:
# 2) Setup MD simulation, minimize, and equilibrate
integrator = LangevinIntegrator(277*kelvin, 1/picosecond, 0.002*picoseconds)    
simulation = Simulation(modeller.topology, system, integrator)
platform = simulation.context.getPlatform()
print(f"Simulation running with {platform.getName()}")
simulation.context.setPositions(modeller.positions)

print("Minimizing... ", end="", flush=True)
simulation.minimizeEnergy()
print("done", flush=True)

Simulation running with OpenCL
Minimizing... done


In [6]:
# Reallocate the velocity, another "equillibriate time"
simulation.context.setVelocitiesToTemperature(277*kelvin)

statereporter = StateDataReporter(stdout, 50000, step=True, time=True, volume=True, totalEnergy=True, temperature=True, elapsedTime=True)
simulation.reporters.append(statereporter)
simulation.step(eqtime)
print("Equilibrating... done", flush=True)
simulation.reporters = []

#"Step","Time (ps)","Total Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Elapsed Time (s)"
50000,99.99999999994834,10820.017575599253,272.9920535821453,729.3159456321972,0.0004208087921142578
100000,200.00000000022686,10650.262529300526,269.9060828446216,729.3159456321972,20.366405725479126
150000,300.00000000070435,10674.070968327578,274.08510062561476,729.3159456321972,41.441725969314575
200000,400.00000000118183,11043.480946414173,277.1887970271424,729.3159456321972,62.508070945739746
250000,500.0000000016593,11309.745137240738,287.2715107701972,729.3159456321972,83.64516592025757
300000,599.9999999996356,10878.915440037847,280.381540614316,729.3159456321972,104.72516703605652
350000,699.999999997271,11144.249186750501,285.00120006038907,729.3159456321972,125.86165404319763
400000,799.9999999949063,10819.482116993517,280.0819704936588,729.3159456321972,146.93242383003235
450000,899.9999999925416,10845.288577603176,274.3757574874,729.3159456321972,168.27917098999023
500000

In [7]:
#save the checkpoint, everything is inside
simulation.saveCheckpoint("equilibrated.chkpt")

In [8]:
# 3) Run production simulation

print("Production run... ", end="", flush=True)

simulation.loadCheckpoint("equilibrated.chkpt")

simulation.context.setVelocitiesToTemperature(277*kelvin)
statereporter = StateDataReporter("traj.csv", 50000, step=True, time=True, volume=True, totalEnergy=True, temperature=True, elapsedTime=True)
trajreporter  = HDF5Reporter(f"traj.h5", 500)
simulation.reporters.append(statereporter)
simulation.reporters.append(trajreporter)
simulation.step(50000000)
trajreporter.close()
print("done", flush=True)

Production run... done
